In [1]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt   
plt.style.use('seaborn-whitegrid')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from BabyMAKROModel import BabyMAKROModelClass

C:\Users\nbk863\AppData\Local\Temp\ipykernel_8452\3413739869.py:10: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


# Setup

In [2]:
model = BabyMAKROModelClass(name='baseline') # create an instance of the model

In [3]:
par = model.par
ss = model.ss
sol = model.sol

In [ ]:
ss.r

# Find steady state

In [ ]:
model.find_ss(do_print=True)

# Speed and error tests

In [ ]:
model.set_exo_ss()
model.set_unknowns_ss()

**Python evaluation:**

In [ ]:
%time model.evaluate_blocks(py=True)

**Check errors:**

In [ ]:
errors = model.get_errors(do_print=True)

In [ ]:
if np.allclose(errors,0.0):
    print('errors are zero')
else:
    raise ValueError('errors are NOT zero')

**First numba evaluation:**

In [ ]:
%time model.evaluate_blocks()

**Time:**

In [ ]:
%timeit model.evaluate_blocks()
%timeit model.get_errors()

In [ ]:
%timeit model.evaluate_blocks(py=True)